## word2vec 

In [1]:
import gensim

In [13]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec

In [5]:
# news data 불러오기
import pandas as pd
news=pd.read_csv('./news_tb.csv')
len(news)

21761

In [7]:
news.head()

,Unnamed: 0,ID,CmpID,Year,Title,Content
0,0,1,1129,2021,"포스코ICT, 카카오워크 통해 RPA 서비스 시작",포스코 의 업무자동화 소프트웨어 로봇인 솔루션과 카카오워크간 연동 서비스가 시작된다...
1,1,2,218,2021,"그리티, 특별관계자 지분변동",그리티는 문영우 및 특별관계자의 지분율이 에서 로 변동했다고 일 공시했다 한편 그리...
2,2,3,1129,2021,"포스코ICT, 2021 기술 컨퍼런스 개최",포스코대표 정덕균가 지난 일 올해 추진된 기술개발 성과를 공유하고 최신 기술동향과 ...
3,3,4,2,2021,"[코스닥 대표 인터뷰]임형섭 석경에이티 대표 ""25년 나노 소재 노하우로 5G 시장...",파이낸셜뉴스우리나라에서 년 처음 시작해 년 간 나노 소재를 하고 있습니다 등의 가치...
4,4,5,218,2021,"그리티, 특별관계자 지분변동",그리티는 문영우 및 특별관계자의 지분율이 에서 로 변동했다고 일 공시했다 한편 그리...


In [6]:
print(news.isnull().values.any()) #결측값 없음, 숫자, 영어 제거된 상태

False


In [14]:
#로컬에 저장한 mecab 파일 경로 지정해주기
# 형태소분석기 okt 활용 토큰화 작업
 
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

tokenized_data = []
for sentence in tqdm(news['Content']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    #stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    #tokenized_data.append(stopwords_removed_sentence)
    tokenized_data.append(tokenized_sentence)

100%|██████████| 21761/21761 [25:02<00:00, 14.48it/s]  


In [27]:
# 토큰화된 뉴스 데이터 학습시키기
#https://hoonzi-text.tistory.com/2 파라미터 정리 
# size : 벡터 차원수 ,
# window : 훈련시 앞 뒤로 고려하는 단어의 개수
# min_count :최소빈도수 , 해당 빈도수보다 작게 등장한 단어 제외
# worker : 병렬처리할 스레드 수 
#sg : 분석방법론 , 1 = skip-gram, 0=cbow
#iter 학습횟수

from gensim.models import Word2Vec
model=Word2Vec(tokenized_data, vector_size=100, window=3, min_count=5,workers=4, sg=0,epochs=100)

In [28]:
model.wv.vectors.shape

(27429, 100)

In [ ]:
e_친환경=model.wv.most_similar("친환경",topn=50)
e_환경=model.wv.most_similar("환경",topn=50)
s_사회=model.wv.most_similar("사회",topn=50)
s_근로=model.wv.most_similar("근로",topn=50)
g_경영=model.wv.most_similar("경영",topn=50)
g_지배=model.wv.most_similar("지배",topn=50)


In [48]:
#e=e_친환경+e_환경
#s=s_근로+s_사회
#g=g_지배+g_경영

def make_dict(df):
    temp = []
    for word, percent in df:
        temp.append(word)
    df = temp
    return df

e_친환경df=make_dict(e_친환경)
e_환경df=make_dict(e_환경)
s_근로df=make_dict(s_근로)
s_사회df=make_dict(s_사회)
g_지배df=make_dict(g_지배)
g_경영df=make_dict(g_경영)

In [50]:
esg_df=pd.DataFrame(zip(e_친환경df, e_환경df,s_근로df,s_사회df,g_지배df,g_경영df))
esg_df.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df.to_csv('./esg_dict_ver1.csv')